In [3]:
from ResidualAttentionNetwork import ResidualAttentionNetwork
import os, shutil
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import array_to_img

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from PIL import Image
import cv2
import shutil

In [4]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [5]:
#get working directory
os.getcwd()

'C:\\Users\\Mojtaba Jafari Tadi\\Projects_AMK\\Radiology Project\\fatpad'

In [6]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'brucechou1983_CheXNet_Keras_0.3.0_weights.h5',
 'CNN_automated_medical_diagnostics_five_layers_chopped_two_classes_trained_on_twenty_epochs_Adam_0.0001.ipynb',
 'CNN_FATPAD_MultiHeadAttention.ipynb',
 'CNN_FATPAD_ResidualAttention.ipynb',
 'CNN_FATPAD_SimplePlainNN.ipynb',
 'CNN_FATPAD_TransferLearning-DenseNet.ipynb',
 'CNN_FATPAD_TransferLearning-EfficientNet.ipynb',
 'CNN_FATPAD_TransferLearning-Inception.ipynb',
 'CNN_FATPAD_TransferLearning-NASNet.ipynb',
 'CNN_FATPAD_TransferLearning-ResNet.ipynb',
 'CNN_FATPAD_TransferLearning-Xception.ipynb',
 'CNN_FATPAD_TransferLearning.ipynb',
 'Labels_FatPadDetection.xlsx',
 'Prepare_Xray_Data.ipynb',
 'README.md',
 'ResidualAttentionNetwork.py',
 '__pycache__']

In [7]:
#change working directory to where the datasets are
root_directory=os.chdir('C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project')

In [8]:
#Provide train, validation, and test EFFUSION + FRACTURE on LATERAL SIDE
base_dir_train = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_LateralSplit/train'
base_dir_validate = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_LateralSplit/validation'
#base_dir_test = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/test'

print(base_dir_train)

C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_LateralSplit/train


In [22]:
#run tensorflow-keras ImageDataGenerator on the directory which contains train, validate and test

# The validation data should not be augmented (just rescaled)
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=1
IMAGE_SHAPE=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)
batch_size=128
epochs = 500
num_classes = 2

train_datagen = ImageDataGenerator( rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=.33)


#test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        base_dir_train,
       shuffle=True,
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    color_mode='grayscale',
    batch_size=batch_size)

validation_generator = train_datagen.flow_from_directory(
        base_dir_validate, 
       target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode="categorical",
    color_mode='grayscale',
    shuffle=True)

Found 790 images belonging to 2 classes.
Found 223 images belonging to 2 classes.


In [23]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

    

In [24]:
train_generator.image_shape
##batch = train_generator.next()
#image = batch[0].astype('float32')
#image.shape
#pyplot.imshow(image[8].reshape(180,180), cmap=pyplot.get_cmap('gray'))

(128, 128, 1)

In [25]:
from matplotlib import pyplot
gr=0
if gr: 
    fig, axs = plt.subplots(3,3, figsize=(15, 15), facecolor='w', edgecolor='k')
    axs = axs.ravel()
    for i in range(9):
        # define subplot
        # generate batch of images
        batch = train_generator.next()
        # convert to unsigned integers for viewing
        image = batch[0]
        # plot raw pixel data
        axs[i].imshow(image[i].reshape(180,180), cmap=pyplot.get_cmap('gray'))# show the figure


In [26]:
#load the root model and load trained weights into it
print(train_generator.n)
steps_per_epoch = int(train_generator.n/32)
print(steps_per_epoch)
validation_steps=int(validation_generator.n/32)
print(validation_steps)

790
24
6


In [27]:
# should add smoothing to these plots
def plot_history(h):
    acc = h.history['acc']
    val_acc = h.history['val_acc']
    loss = h.history['loss']
    val_loss = h.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [28]:
checkpoint_cb = tensorflow.keras.callbacks.ModelCheckpoint("xray_model.h5", save_best_only=True)

early_stopping_cb = tensorflow.keras.callbacks.EarlyStopping(
    patience=50, restore_best_weights=True
)

In [29]:
initial_learning_rate = 0.015
lr_schedule = tensorflow.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

In [30]:
model = ResidualAttentionNetwork(input_shape=IMAGE_SHAPE,n_classes=2,activation='softmax').build_model()
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_50 (Conv2D)              (None, 128, 128, 32) 320         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 64, 64, 32)   0           conv2d_50[0][0]                  
__________________________________________________________________________________________________
batch_normalization_45 (BatchNo (None, 64, 64, 32)   128         max_pooling2d_5[0][0]            
____________________________________________________________________________________________

In [31]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

In [32]:
train_generator.class_indices


{'EFFR_cl0': 0, 'EFFR_cl1': 1}

In [33]:

METRICS = [
    tensorflow.keras.metrics.BinaryAccuracy(),
    tensorflow.keras.metrics.Precision(name="precision"),
    tensorflow.keras.metrics.Recall(name="recall"),
]
model.compile(optimizer=optimizers.Adam(lr=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'],
)


history = model.fit(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=500,
    validation_data=validation_generator,
    validation_steps=STEP_SIZE_VALID,
    callbacks=[checkpoint_cb])

Epoch 1/500
6/6 [==============================] - 74s 11s/step - loss: 2.9887 - accuracy: 0.5147 - val_loss: 0.6902 - val_accuracy: 0.6094
Epoch 2/500
6/6 [==============================] - 69s 13s/step - loss: 2.9315 - accuracy: 0.4988 - val_loss: 0.6907 - val_accuracy: 0.5781
Epoch 3/500
6/6 [==============================] - 70s 11s/step - loss: 2.0911 - accuracy: 0.4761 - val_loss: 0.6913 - val_accuracy: 0.5156
Epoch 4/500
6/6 [==============================] - 71s 11s/step - loss: 1.3023 - accuracy: 0.4703 - val_loss: 0.6931 - val_accuracy: 0.4453
Epoch 5/500
6/6 [==============================] - 81s 13s/step - loss: 0.8775 - accuracy: 0.5428 - val_loss: 0.6924 - val_accuracy: 0.4922
Epoch 6/500
6/6 [==============================] - 70s 11s/step - loss: 0.7641 - accuracy: 0.5024 - val_loss: 0.6919 - val_accuracy: 0.5469
Epoch 7/500
6/6 [==============================] - 70s 11s/step - loss: 0.7676 - accuracy: 0.4536 - val_loss: 0.6939 - val_accuracy: 0.4609
Epoch 8/500
6/6 [===

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=20)
test_acc